In [10]:
######## Create, Train, and Predict Models

# Loads train, test, and val datasets
# Instantiates Models
# Trains Models
# Predicts using Models

Import All Required Modules

In [3]:
import sys

# Manually add the project root to sys.path
sys.path.append('/Users/joaquinuriarte/Documents/GitHub/sports-betting/')

# === STEP 0: Imports
print("# === STEP 0: Imports: Starting ...")
from modules.model_manager.trainer.trainer import Trainer
from modules.model_manager.predictor.predictor import Predictor
from modules.model_manager.factories.model_factory import ModelFactory
from modules.model_manager.helpers.configuration_loader import ConfigurationLoader as MMConfigLoader
from modules.model_manager.model_manager import ModelManager
print("# === STEP 0: Imports: Complete")

# === STEP 0: Imports: Starting ...


2025-01-22 18:57:54.437477: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# === STEP 0: Imports: Complete


In [4]:
# === STEP 1: Dependency Instantiations And Global Variable Declarations
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...")
## === MODEL MANAGER
yaml_path0 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_000.yaml'
yaml_path1 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_001.yaml'
yaml_path2 = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/configs/model_v01/model_v01_002.yaml'

yaml_paths = [yaml_path0, yaml_path1]


checkpoint = '/Users/joaquinuriarte/Documents/GitHub/sports-betting/models'
trainer = Trainer(checkpoint)
predictor = Predictor()
model_factory = ModelFactory()
mm_configuration_loader = MMConfigLoader()
print("# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete")

# === STEP 1: Dependency Instantiations And Global Variable Declarations: Starting ...
# === STEP 1: Dependency Instantiations And Global Variable Declarations: Complete


Load Train, Val, and Test Datasets into Memory

In [5]:
import pickle
import os

# File path to load train, test, and val datasets
train_test_val_dataset_path = "/Users/joaquinuriarte/Documents/GitHub/sports-betting/processed_datasets/model_v01"

train_dataset_path = train_test_val_dataset_path + "/train.pkl"
val_dataset_path = train_test_val_dataset_path + "/val.pkl"
# test_dataset_path = train_test_val_dataset_path + "/test.pkl"

with open(train_dataset_path, "rb") as f:
    train_dataset = pickle.load(f)
    print("Train dataset leaded to memory")
with open(val_dataset_path, "rb") as f:
    validation_dataset = pickle.load(f)
    print("Val dataset leaded to memory")
# with open(test_dataset_path, "rb") as f:
#     test_dataset = pickle.load(f)
#     print("Test dataset leaded to memory")

Train dataset leaded to memory
Val dataset leaded to memory


Create and Train Models

In [6]:
# === STEP 4: MODEL MANAGER
print("# === STEP 4: MODEL MANAGER: Starting ...")
model_manager = ModelManager(trainer, predictor, model_factory, mm_configuration_loader)
models = model_manager.create_models([yaml_path2])

# === STEP 4: MODEL MANAGER: Starting ...


In [7]:
train_dataset.examples[0].features
len(train_dataset.examples[0].features) # should be 178

178

In [8]:
# If no val dataset, code requires to push None instead
val_dataset = validation_dataset if validation_dataset is not None else None

model_manager.train(models, [(train_dataset, val_dataset)], save_after_training=True)
print("# === STEP 4: MODEL MANAGER: Complete")

INFO:root:Training model '39cf74dd0ecb478f456add01f6470ccc' for 20 epochs with batch size 32.


Epoch 1/20
557/557 [==============================] - 14s 22ms/step - loss: 375.9384 - mse: 375.9384 - mae: 13.7983 - val_loss: 259.9140 - val_mse: 259.9140 - val_mae: 13.1998
Epoch 2/20
557/557 [==============================] - 12s 21ms/step - loss: 178.2905 - mse: 178.2905 - mae: 10.6121 - val_loss: 217.0804 - val_mse: 217.0804 - val_mae: 11.7003
Epoch 3/20
557/557 [==============================] - 11s 20ms/step - loss: 175.1241 - mse: 175.1241 - mae: 10.5087 - val_loss: 166.8311 - val_mse: 166.8311 - val_mae: 10.2494
Epoch 4/20
557/557 [==============================] - 12s 21ms/step - loss: 171.6797 - mse: 171.6797 - mae: 10.4029 - val_loss: 169.0943 - val_mse: 169.0943 - val_mae: 10.3727
Epoch 5/20
557/557 [==============================] - 12s 21ms/step - loss: 167.0857 - mse: 167.0857 - mae: 10.2713 - val_loss: 174.4041 - val_mse: 174.4041 - val_mae: 10.5856
Epoch 6/20
557/557 [==============================] - 11s 19ms/step - loss: 166.3196 - mse: 166.3196 - mae: 10.2448 - va

INFO:root:Model '39cf74dd0ecb478f456add01f6470ccc': Finished training.


Model saved successfully at: models/39cf74dd0ecb478f456add01f6470ccc/model_weights_39cf74dd0ecb478f456add01f6470ccc.pth
# === STEP 4: MODEL MANAGER: Complete


Accuracy When Predicting Game Winner

In [41]:
predictions = []
for model in models:
    print(model)
    predictions.append(model.predict(validation_dataset.examples, return_target_labels=True))

In [42]:
total_examples = 0
correct_predictions = 0
correct_b_predictions = 0

for row in predictions[0].itertuples(index=False):
    total_examples += 1
    
    # Compare predictions and target labels for the current feature
    if (row.final_score_A > row.final_score_B and row.target_final_score_A > row.target_final_score_B):
        correct_predictions += 1
    elif (row.final_score_A < row.final_score_B and row.target_final_score_A < row.target_final_score_B):
        correct_predictions += 1
        correct_b_predictions += 1  

# Calculate accuracy
accuracy = correct_predictions / total_examples
percent_of_b_in_total_correct_pred = correct_b_predictions / correct_predictions
print(f"Accuracy across all features on validation dataset: {accuracy} and percent_of_b_in_total_correct_pred: {percent_of_b_in_total_correct_pred}")

Accuracy across all features on validation dataset: 0.6038773906209065 and percent_of_b_in_total_correct_pred: 0.005206073752711497


In [10]:
%tensorboard --logdir logs/fit/39cf74dd0ecb478f456add01f6470ccc

UsageError: Line magic function `%tensorboard` not found.
